<a href="https://colab.research.google.com/github/cbenaventte/Cymatics/blob/main/Cimatica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la Cimática y Simulación de Figuras de Chladni

## ¿Qué es la Cimática?
La **cimática** es el estudio de los patrones visibles que se forman cuando un medio (como arena, sal o agua) es sometido a vibraciones sonoras. Fue popularizado por Hans Jenny en el siglo XX, pero sus raíces se remontan a los experimentos de Ernst Chladni en el siglo XVIII. Chladni usó placas metálicas cubiertas de arena y las hizo vibrar con un arco de violín, observando cómo las partículas se acumulaban en líneas donde la vibración era mínima (**nodos**), formando figuras geométricas conocidas como **figuras de Chladni**.

Estas figuras dependen de la frecuencia de vibración y de las propiedades de la placa (como su forma, tamaño y material). En este documento, simulamos estas figuras para una placa cuadrada de acero usando Python.

---

## Descripción del Código
El código simula figuras de Chladni en una placa cuadrada de acero de 20 cm de lado. Genera partículas que representan arena y las coloca en los nodos de vibración, mostrando cómo cambian los patrones al variar los modos de vibración ($m$, $n$). Además, calcula y muestra la frecuencia en hercios (Hz) para cada modo.

### Parámetros de la Placa
- Longitud del lado: $a = 0.2 \, \text{m}$ (20 cm)
- Espesor: $h = 0.001 \, \text{m}$ (1 mm)
- Módulo de Young (acero): $E = 200 \times 10^9 \, \text{Pa}$
- Coeficiente de Poisson: $\nu = 0.3$
- Densidad: $\rho = 7850 \, \text{kg/m}^3$
- Número de partículas: 5000

---

## Fórmulas Principales

1. **Rigidez a la Flexión ($D$)**  
   La rigidez de la placa se calcula como:
   $$
   D = \frac{E h^3}{12 (1 - \nu^2)}
   $$
   Donde $D$ mide la resistencia de la placa a la flexión y depende del material y el espesor.

2. **Velocidad Efectiva ($c_{\text{effective}}$)**  
   Esta constante relaciona las propiedades del material con la propagación de las ondas:
   $$
   c_{\text{effective}} = \sqrt{\frac{D}{\rho h}}
   $$

3. **Frecuencia de Vibración ($f_{mn}$)**  
   La frecuencia en Hz para cada modo $(m, n)$ en una placa cuadrada con bordes simplemente apoyados es:
   $$
   f_{mn} = \frac{\pi}{2} c_{\text{effective}} \left( \left( \frac{m}{a} \right)^2 + \left( \frac{n}{a} \right)^2 \right)
   $$
   Aquí, $m$ y $n$ son números enteros positivos que indican el número de nodos en las direcciones $x$ e $y$.

4. **Amplitud de Vibración ($U(x, y)$)**  
   La forma de las figuras de Chladni está dada por:
   $$
   U(x, y) = \sin\left( \frac{m \pi x}{a} \right) \sin\left( \frac{n \pi y}{a} \right)
   $$
   Las partículas se acumulan en los **nodos**, donde $U(x, y) \approx 0$.

---

## ¿Cómo Funciona el Código?
1. **Cálculo de Frecuencias**: Genera una lista de modos $(m, n)$ (de 1 a 5) y calcula sus frecuencias usando la fórmula de $f_{mn}$.
2. **Simulación de Partículas**: Coloca 5000 partículas aleatoriamente en la placa y muestra solo las que están cerca de los nodos (donde $|U| < 0.05$).
3. **Interactividad**: Usa un deslizador para cambiar entre modos y ver cómo varían los patrones y las frecuencias.

Por ejemplo, para el modo fundamental ($m = 1$, $n = 1$):
- $f_{11} \approx 120 \, \text{Hz}$ (verificado con los parámetros dados).

Ejecuta el código en este notebook para explorar los patrones de Chladni de forma interactiva.

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from IPython.display import clear_output

# Parámetros de la placa
a = 0.2  # Longitud del lado en metros (20 cm)
h = 0.001  # Espesor en metros (1 mm)
E = 200e9  # Módulo de Young en Pa (acero)
nu = 0.3  # Coeficiente de Poisson (acero)
rho = 7850  # Densidad en kg/m³ (acero)
num_particles = 5000  # Número de partículas de arena

# Calcular la rigidez a la flexión D
D = E * h**3 / (12 * (1 - nu**2))

# Calcular la velocidad efectiva para las ondas flexurales
c_effective = np.sqrt(D / (rho * h))

# Lista de modos (m, n) y sus frecuencias en Hz
modes = []
for m in range(1, 6):
    for n in range(1, 6):
        f_mn = (np.pi / 2) * c_effective * ((m / a)**2 + (n / a)**2)  # Convertir a Hz
        modes.append((m, n, f_mn))

# Ordenar por frecuencia
modes.sort(key=lambda x: x[2])

# Función para graficar el patrón de Chladni con partículas
def plot_chladni(index):
    clear_output(wait=True)
    m, n, f_mn = modes[index]

    # Generar posiciones aleatorias para las partículas
    np.random.seed(0)  # Para reproducibilidad
    particles_x = np.random.uniform(0, a, num_particles)
    particles_y = np.random.uniform(0, a, num_particles)

    # Calcular la amplitud U en las posiciones de las partículas
    U_particles = np.sin(m * np.pi * particles_x / a) * np.sin(n * np.pi * particles_y / a)

    # Filtrar partículas que están cerca de los nodos (U ≈ 0)
    threshold = 0.05  # Umbral para considerar que está cerca del nodo
    near_nodes = np.abs(U_particles) < threshold

    # Crear una malla para el fondo de amplitud
    x = np.linspace(0, a, 100)
    y = np.linspace(0, a, 100)
    X, Y = np.meshgrid(x, y)
    U = np.sin(m * np.pi * X / a) * np.sin(n * np.pi * Y / a)

    # Graficar
    plt.figure(figsize=(6, 6))

    # Fondo de amplitud oscuro
    plt.contourf(X, Y, U, levels=20, cmap='Blues_r', alpha=0.5)  # Azul oscuro invertido

    # Partículas brillantes
    plt.scatter(particles_x[near_nodes], particles_y[near_nodes], s=2, color='gold')  # Dorado brillante

    # Configuración del gráfico
    plt.title(f'Figura de Chladni para m={m}, n={n}, f={f_mn:.2f} Hz')
    plt.xlabel('x (m)')
    plt.ylabel('y (m)')
    plt.xlim(0, a)
    plt.ylim(0, a)
    plt.gca().set_facecolor('#1a2a44')  # Fondo oscuro azul-negro
    plt.gca().tick_params(axis='both', colors='lightgrey')  # Marcas en gris claro

    plt.show()

# Deslizador interactivo para elegir el modo
interact(plot_chladni,
         index=IntSlider(min=0, max=len(modes)-1, step=1, value=0, description='Modo'))

interactive(children=(IntSlider(value=0, description='Modo', max=24), Output()), _dom_classes=('widget-interac…

<function __main__.plot_chladni(index)>